# Making Predictions: An Example Using the Beer Dataset

We will go through the proccess for predicting Y quantities with SIMCA-Q.

We will use a SIMCA project where two OPLS models have been built using the BEER dataset typically used in SIMCA courses. The picture below shows the initial observations and variables of the original dataset used in this project:

![Original Dataset](Images/Dataset1.png)

This SIMCA file is available from this [repository folder](https://github.com/OEM-Sartorius-Data-Analytics/SimcaQ_Python_Scripting_Guide/tree/main/06_PredictionInterface_1) as *BEER_NIR_alcohol_example.usp*.

The SIMCA project file has two OPLS models, both predicting the alcohol content:

![Models](Images/Models.png)

We will show in this example how to predict the alcohol content of a spectrum not used to build the model. This spectrum will be in a csv format with two rows, one for the variable names and one for the variable values. When opened in Excel it looks like:

![Spectrum](Images/InputFile.png)

This csv file is available from this repository folder as [predictionDataFile2.csv](predictionDataFile2.csv).

## Accessing the SIMCA-Q COM Interface

We will first need to access the SIMCA-Q COM interface. For this we will follow the [approach detailed in the guide](https://github.com/OEM-Sartorius-Data-Analytics/SimcaQ_Python_Scripting_Guide/blob/main/00_COM_and_License/COM_and_License.md)

In [1]:
def dispatch(app_name:str):
    try:
        from win32com import client
        app = client.gencache.EnsureDispatch(app_name)
    except AttributeError:
        # Corner case dependencies.
        import os
        import re
        import sys
        import shutil
        # Remove cache and try again.
        MODULE_LIST = [m.__name__ for m in sys.modules.values()]
        for module in MODULE_LIST:
            if re.match(r'win32com\.gen_py\..+', module):
                del sys.modules[module]
        shutil.rmtree(os.path.join(os.environ.get('LOCALAPPDATA'), 'Temp', 'gen_py'))
        from win32com import client
        app = client.gencache.EnsureDispatch(app_name)
    return app

In [2]:
simcaq = dispatch('Umetrics.SIMCAQ')

If we print the variable *simcaq* we can see that it is an *ISIMCAQ* object:

In [3]:
print(simcaq)

<win32com.gen_py.SIMCA-Q 17 Type Library.ISIMCAQ instance at 0x1970782558512>


## Opening the SIMCA Project

We can use the *ISIMCAQ* method *OpenProject()* to open the SIMCA Project *BEER_NIR_alcohol_example.usp*:

In [4]:
pathSimcaProject = 'BEER_NIR_alcohol_example.usp'
project = simcaq.OpenProject(pathSimcaProject, "")

If we print the variable *project* we can see that it is an *IProject* object:

In [5]:
print(project)

<win32com.gen_py.SIMCA-Q 17 Type Library.IProject instance at 0x1970782556640>


As [discussed](https://github.com/OEM-Sartorius-Data-Analytics/SimcaQ_Python_Scripting_Guide/blob/main/01_ProjectInterface/ExploreProjectInterface.md), we can retrieve different attributes of the project straigtht away from this object.

For instance, the name of the project:

In [6]:
projectName = project.GetProjectName() 
print(projectName)

BEER_NIR_alcohol_example


The number of models within the project:

In [7]:
numberModels = project.GetNumberOfModels()
print(numberModels)

2


The number of datasets:

In [8]:
numberDatasets = project.GetNumberOfDatasets()
print(numberDatasets)

2


The project has indeed two databases, the one shown above and another one containing the 1st derivative of the spectra, which was used to build the second model:

![1st Derivative Dataset](Images/Dataset2.png)

## Accessing a Model

As [discussed](https://github.com/OEM-Sartorius-Data-Analytics/SimcaQ_Python_Scripting_Guide/blob/main/04_ModelInterface_0/ModelInterface_Introduction.md), we can access models by using the *IProject* method *GetModel()*, which receives as an input parameter.

In the example below we retrieve handles for all the models in the project and store them in a list

In [9]:
oModelList = []
for iModel in range(1,numberModels+1):
    modelNumber = project.GetModelNumberFromIndex(iModel)
    oModelList.append(project.GetModel(modelNumber))

We can check that effectively this created a list of *IModel* objects:

In [10]:
for iModel, oModel in enumerate(oModelList):
    print(oModel)

<win32com.gen_py.SIMCA-Q 17 Type Library.IModel instance at 0x1970783442768>
<win32com.gen_py.SIMCA-Q 17 Type Library.IModel instance at 0x1970783442432>


We can e.g., check the name and title of the loaded models using methods available for *IModel* objects:

In [11]:
for iModel, oModel in enumerate(oModelList):
    print(f'Model[{iModel+1}] -> name: {oModel.GetName()}, title: {oModel.GetTitle()}')

Model[1] -> name: M1, title: Alcohol Model
Model[2] -> name: M3, title: Alcohol Model - 1st Derivative Data


## Prepare Predictions

The next step for making predictions for new data is to create an *IPreparePrediction* object for the model of interest.

We will first focus of the first model i.e., M1:

In [12]:
oPrepPredList = []
for iModel, oModel in enumerate(oModelList):
    oPrepPredList.append(oModel.PreparePrediction())

We can check that effectively this created a list of *IPreparePrediction* objects:

In [13]:
for iPrepPred, oPrepPred in enumerate(oPrepPredList):
    print(oPrepPred)

<win32com.gen_py.SIMCA-Q 17 Type Library.IPreparePrediction instance at 0x1970783450304>
<win32com.gen_py.SIMCA-Q 17 Type Library.IPreparePrediction instance at 0x1970783446560>


Let's have a look now at the variables used for predicition in both models. We start with model M1:

In [14]:
predictionVariablesM1 = oPrepPredList[0].GetVariablesForPrediction()
numberPredictionVariablesM1 = predictionVariablesM1.GetSize()
print(numberPredictionVariablesM1)

1051


Let's print the names of the first 10 variables needed to make predictions with this model:

In [15]:
for iVar in range(1,11):
    print(f'Variable in position {iVar}: {predictionVariablesM1.GetVariable(iVar).GetName(1)}')

Variable in position 1: Alcohol
Variable in position 2: 400
Variable in position 3: 402
Variable in position 4: 404
Variable in position 5: 406
Variable in position 6: 408
Variable in position 7: 410
Variable in position 8: 412
Variable in position 9: 414
Variable in position 10: 416


But you can already guess that *Alcohol* is a Y quantity, indeed the one that we want to predict. We can print instead only the X variables by using the *IsXVariable()* method: 

In [16]:
for iVar in range(1,11):
    if predictionVariablesM1.GetVariable(iVar).IsXVariable()==1:
        print(f'Variable in position {iVar}: {predictionVariablesM1.GetVariable(iVar).GetName(1)}')

Variable in position 2: 400
Variable in position 3: 402
Variable in position 4: 404
Variable in position 5: 406
Variable in position 6: 408
Variable in position 7: 410
Variable in position 8: 412
Variable in position 9: 414
Variable in position 10: 416


Now we need to populate the *IPreparePrediction* object with the data for prediction i.e., a spectrum not used to build the model. As indicated above, in this example this spectrum is available as a 2 rows csv file, the first row indicating the names of the variables and the second row containing quantitative values for these variables. The way to load input files will be dependent on the format/structure of these files, so a general way cannot be proposed. For the format of our file, the following function will work, which will return two lists, one for the name of the variables and one for their values:

In [17]:
import pandas as pd

def LoadInputCSVFile(predictionDataFile):
    pred_sample = pd.read_csv(predictionDataFile, index_col=False)
    inputVariableNames = list(pred_sample.columns.values)
    inputData = pred_sample.iloc[0,:].to_list()
    return inputVariableNames, inputData

predictionDataFile = "predictionDataFile2.csv"
inputVariableNames, inputData = LoadInputCSVFile(predictionDataFile)

We can print the first 10 components of both lists and verify that they coincide with the caption shown above for the file opened in Excel:

In [18]:
for iVar in range(0,10):
    print(f'{inputVariableNames[iVar]}: {inputData[iVar]}')

400: 0.0718325
402: 0.073309
404: 0.0727075
406: 0.072502
408: 0.072041
410: 0.071419
412: 0.070675
414: 0.070036
416: 0.069309
418: 0.068394


To populate the *IPreparePrediction* object we can follow the [approach discussed in a previous section](https://github.com/OEM-Sartorius-Data-Analytics/SimcaQ_Python_Scripting_Guide/blob/main/06_PredictionInterface_0/PredictionInterface_Introduction.md). Basically, we first create a dictionary with the names of the variables used to build the models as keys, and the position of these variables in the *IVariableVector* object returned by the *IPreparePrediction* method *GetVariablesForPrediction()* as values:

In [19]:
variables_vec = [predictionVariablesM1.GetVariable(i+1).GetName(1) for i in range(predictionVariablesM1.GetSize())]
NameLookup = {name: ix+1 for ix, name in enumerate(variables_vec)}

Now we can iterate over the data lists for prediction, and provide SIMCA-Q with only the variables whose names coincide with those used to build the model, and also in the correct order:

In [20]:
iObs = 1 # We only have 1 observation in the file with data for prediction
for i, name in enumerate(inputVariableNames):
    if name in NameLookup:
        oPrepPredList[0].SetQuantitativeData(iObs, NameLookup[name], inputData[i])

## Making Predictions

Now we can create *IPrediction* objects that will allow us to handle the predicted quantities themselves:

In [21]:
oPredictionM1 = oPrepPredList[0].GetPrediction()

With this object we can retrieve any predicted quantity.

### Predicting the alcohol content

To retrieve a handle for the predicted Alcohol content, using all predictive scores (in this case is just 1):

In [22]:
numPredictiveScores = oModelList[0].GetNumberOfPredictiveComponents()
hPredictedY = oPredictionM1.GetYPredPS(numPredictiveScores,True,True,None)

And to retrieve the actual value for the alcohol content:

In [23]:
predictionDataMatrix = hPredictedY.GetDataMatrix()
iObs = 1
iVarY = 1
predictedY = predictionDataMatrix.GetData(iObs,iVarY)
print(f'Alcohol Content: {predictedY}')

Alcohol Content: 2.850982189178467
